Install requirements.

In [ ]:
!pip install -r requirements.txt -qq

Download a wandb model below.

In [ ]:
import wandb

# define wandb environment
%env WANDB_LOG_MODEL=true
%env WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1

In [ ]:
# download wandb model
import wandb
run = wandb.init()
artifact = run.use_artifact('oumar-kane-team/bart-custom-train-fw-translation-bayes-hpsearch-v5/model_hnkyvzn:v0', type='dataset')
artifact_dir = artifact.download()

In [ ]:
wandb.finish()

In [ ]:
# transfer the model inside the artifact to data/checkpoints/name_of_model
def transfer_model(artifact_dir, model_name):
    import os
    import shutil
    import glob

    # transfer the model inside the artifact to data/checkpoints/name_of_model
    os.makedirs(f'data/checkpoints/{model_name}', exist_ok=True)
    for file in glob.glob(f'{artifact_dir}/*'):
        shutil.copy(file, f'data/checkpoints/{model_name}')

    # delete the artifact
    shutil.rmtree(artifact_dir)

# transfer the model inside the artifact to data/checkpoints/name_of_model
transfer_model(artifact_dir, 'fw_bart_custom_train_v5_checkpoints')